In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

In [2]:
db = database('trfc.db')

In [3]:
app, rt = fast_app()

In [4]:
server = JupyUvi(app)

In [5]:
query = f'''
    SELECT 
            m.manager_name,
            r.*
        FROM results r
        LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
            AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
        LEFT JOIN managers m ON mr.manager_id = m.manager_id
        LEFT JOIN season_league_tiers slt ON r.season = slt.season AND r.game_type = 'League'
'''

In [6]:
df = pd.read_sql(query, db.conn)

In [7]:
def prepare_streaks(df):
    df['is_win'] = df['outcome'] == 'W'
    df['is_unbeaten'] = df['outcome'] != 'L'
    df['is_clean_sheet'] = df['goals_against'] == 0
    df['is_draw'] = df['outcome'] == 'D'
    df['is_winless'] = df['outcome'] != 'W'
    df['is_loss'] = df['outcome'] == 'L'
    df['is_clean_sheet'] = df['goals_against'] == 0
    df['is_win_to_nil'] = (df['outcome'] == 'W') & (df['goals_against'] == 0)
    df['is_loss_to_nil'] = (df['outcome'] == 'L') & (df['goals_for'] == 0)
    return df

In [8]:
def get_streak_lengths(group):
    streak_groups = (group != group.shift()).cumsum()
    return group.groupby(streak_groups).sum()

In [9]:
def calc_streaks(df, focus, condition):
    return df.groupby(focus)[condition].apply(
        lambda x: get_streak_lengths(x).max()
    )

def get_streaks_df(df, focus):
    df = prepare_streaks(df)
    
    index_name = {
        'manager_name': 'Manager',
        'player_name': 'Player',
        'opposition': 'Opponent',
        'season': 'Season'
    }

    return pd.DataFrame({
        'Wins': calc_streaks(df, focus, 'is_win'),
        'Unbeaten': calc_streaks(df, focus, 'is_unbeaten'),
        'Clean Sheets': calc_streaks(df, focus, 'is_clean_sheet'),
        'Wins to nil': calc_streaks(df, focus, 'is_win_to_nil'),
        'Draws': calc_streaks(df, focus, 'is_draw'),
        'Winless': calc_streaks(df, focus, 'is_winless'),
        'Defeats': calc_streaks(df, focus, 'is_loss'),
        'Losses to nil': calc_streaks(df, focus, 'is_loss_to_nil')
    }).reset_index().rename(columns=index_name)

In [10]:
man_streaks = get_streaks_df(df, 'manager_name')

man_streaks.head(3)

,Manager,Wins,Unbeaten,Clean Sheets,Wins to nil,Draws,Winless,Defeats,Losses to nil
0,Alan Rogers,0,0,0,0,0,2,2,1
1,Bert Cooke,7,11,5,5,3,15,7,3
2,Bill Ridding,2,3,0,0,1,11,10,1


In [11]:
opp_streaks = get_streaks_df(df, 'opposition')

opp_streaks.head(3)

,Opponent,Wins,Unbeaten,Clean Sheets,Wins to nil,Draws,Winless,Defeats,Losses to nil
0,AFC Bournemouth,2,10,3,2,3,9,5,3
1,AFC Wimbledon,1,3,1,1,2,3,1,1
2,Accrington Stanley,4,11,3,2,3,6,5,3


In [12]:
ssn_streaks = get_streaks_df(df, 'season')

ssn_streaks.head(3)

,Season,Wins,Unbeaten,Clean Sheets,Wins to nil,Draws,Winless,Defeats,Losses to nil
0,1921/22,2,3,2,1,2,15,6,3
1,1922/23,4,4,2,2,2,11,7,3
2,1923/24,3,11,3,2,3,7,4,3


In [ ]:
def df_to_html(df, table_id='no-id'):    
    return Table(
        Thead(Tr(*[Th(col) for col in df.columns])),
        Tbody(*[Tr(*[Td(row[col]) for col in df.columns]) for idx, row in df.iterrows()]),
        id=table_id
    )

In [ ]:
@rt('/')
def index():
    return (
        H1('Season Streaks'),
        df_to_html(get_streaks_df(df, 'season')[:4], 'ssn-streaks-table'),
        H1('Manager Streaks'),
        df_to_html(get_streaks_df(df, 'manager_name')[:4], 'mgr-streaks-table'),
        H1('Opponent Streaks'),
        df_to_html(get_streaks_df(df, 'opposition')[:4], 'opp-streaks-table')
    )

In [ ]:
# HTMX()

In [ ]:
# server.stop()